# 0. Set up

In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import pickle
from pickle_utils import pickle_load, pickle_dump
from loadGlove import loadGloveModel

In [2]:
captions = pd.read_csv("captions.csv", error_bad_lines=False, warn_bad_lines=False)
print("Skipping bad lines - return to this later")
print(captions.shape)
captions.sample(10, random_state=22)

Skipping bad lines - return to this later
(155392, 3)


,image,above_text,below_text
54053,My Precious Gollum,Hello TARA...,HELLO PRECIOUS
67160,Ecstatic Michael Phelps,tHERE'S A POT OF THE STUFF?,i LOVE POT.
49626,katt williams shocked,What,you actually thought you were getting rp?
9727,Okay Guy,TOOK AN ARROW TO THE KNEE,OKAY..
22047,Rich Men Laughing,and then we told them,their health insurance premiums wouldnt go up
150193,kim jong un,they see me rulin',they hatin'
121915,The Olympic Queen,vodka,NaN
132328,Paperclip,it looks like you're having trouble,fapping to this meme
120444,Honey BooBoo,happy birthday,ali boo boo
15655,Not sure if troll,not sure if nicki minaj,or a mutant from mortal combat


# 1. Load GloVe and save objects for later use

In [3]:
glove_objs = loadGloveModel("glove.txt")

In [ ]:
glove_index_dict = glove_objs[0]
glove_embedding_weights = glove_objs[1]

In [5]:
len(glove_index_dict), glove_embedding_weights.shape

(1917494, (1917494, 300))

In [ ]:
# pickle_dump(glove_objs, "glove_objs.pkl")
# pickle_load("glove_objs.pkl")

# 2. Clean captions and labels

## a. Remove where there are NaNs in the captions or labels

In [ ]:
np.sum(pd.isna(captions))

In [ ]:
captions.iloc[np.where(pd.isna(captions.image))]

In [ ]:
captions = captions[pd.notnull(captions.image)]
captions = captions.replace(np.nan, '', regex=True)

In [ ]:
np.sum(pd.isna(captions))

## b. Everything to lower

In [ ]:
captions['image'] = captions['image'].str.lower()
captions['above_text'] = captions['above_text'].str.lower()
captions['below_text'] = captions['below_text'].str.lower()

In [ ]:
captions.sample(10, random_state=33)

## c. Next

# 3. Creating word mappings

Structure borrowed from Harvard IACS 2019 ComputeFest [code](https://github.com/Harvard-IACS/2019-computefest/blob/master/Friday/data_preprocess.ipnb.ipynb).

In [ ]:
empty = 0 # RNN mask of no data
eos = 1  # end of sentence
start_idx = eos+1 # first real word

def get_idx(vocab, vocabcount):
    word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocab))
    word2idx['<empty>'] = empty
    word2idx['<eos>'] = eos
    
    idx2word = dict((idx,word) for word,idx in word2idx.items())

    return word2idx, idx2word

In [ ]:
# word2idx, idx2word = get_idx(vocab, vocabcount)